In [1]:
%load_ext autotime

In [2]:
### import and basic setup

time: 577 µs


In [3]:
import imp # need to be done for python 3
import logging
imp.reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', 
                    level=logging.DEBUG, datefmt='%I:%M:%S')

time: 6.15 ms


In [4]:
import sys
import os

time: 763 µs


In [5]:
sys.path.append('/Users/pradap/Documents/Research/Python-Package/anhaid/magellan/')

time: 734 µs


In [6]:
import magellan as mg

time: 2.45 s


In [7]:
path_a = os.sep.join(['data', 'DBLP_demo.csv'])
path_b = os.sep.join(['data', 'ACM_demo.csv'])
path_c = os.sep.join(['data', 'dblp_acm_demo_labels.csv'])

time: 1.66 ms


In [8]:
A = mg.read_csv_metadata(path_a, key='id')
B = mg.read_csv_metadata(path_b, key='id')
C = mg.read_csv_metadata(path_c, key='_id', ltable=A, rtable=B, fk_ltable='ltable.id', 
                        fk_rtable='rtable.id')

08:50:51 WARNING:Metadata file is not present in the given path; proceeding to read the csv file.
08:50:51 WARNING:Metadata file is not present in the given path; proceeding to read the csv file.
08:50:51 WARNING:Metadata file is not present in the given path; proceeding to read the csv file.


time: 21.7 ms


In [11]:
feature_table = mg.get_features_for_matching(A, B)

08:58:02 WARNING:Magellan types: id type (str_eq_1w) and id type (numeric) are different.If you want to set them to be same and generate features, update output from get_attr_types and use get_features command.

08:58:02 WARNING:Magellan types: venue type (str_bt_1w_5w) and venue type (str_bt_5w_10w) are different.If you want to set them to be same and generate features, update output from get_attr_types and use get_features command.



time: 57.6 ms


In [12]:
feature_vectors = mg.extract_feature_vecs(C, feature_table=feature_table, attrs_after='gold')

08:58:30 INFO:Required metadata: cand.set key, fk ltable, fk rtable, ltable, rtable, ltable key, rtable key
08:58:30 INFO:Getting metadata from the catalog
08:58:30 INFO:..... Done
08:58:30 INFO:Validating candset key: _id
08:58:30 INFO:..... Done
08:58:30 INFO:Validating foreign key constraint for left table
08:58:30 INFO:..... Done
08:58:30 INFO:Validating foreign key constraint for right table
08:58:30 INFO:..... Done


time: 1.12 s


In [9]:
# create matchers
dt = mg.DTMatcher(name='dt')
rf = mg.RFMatcher(name='rf')
svm = mg.SVMMatcher(name='svm')
lreg = mg.LogRegMatcher(name='logreg')
linreg = mg.LinRegMatcher(name='linreg')
nb = mg.NBMatcher(name='nb')

time: 4.42 ms


In [10]:
help(mg.select_matcher)

Help on function select_matcher in module magellan.matcherselector.mlmatcherselection:

select_matcher(matchers, x=None, y=None, table=None, exclude_attrs=None, target_attr=None, metric='precision', k=5, random_state=None)
    This function selects a matcher from a given list of matchers based on a
    given metric.
    Specifically, this function internally uses scikit-learn's
    cross validation function to select a matcher. There are two ways the
    user can call the fit method. First, interface similar to scikit-learn
    where the feature vectors and target attribute given as projected DataFrame.
    Second, give the DataFrame and explicitly specify the feature vectors
    (by specifying the attributes to be excluded) and the target attribute
    A point to note is all the input parameters have a default value of
    None. This is done to support both the interfaces in a single function.
    
    Args:
        matchers (MLMatcher): List of ML matchers to be selected from.
      

In [18]:
feature_vectors.fillna(0, inplace=True)

time: 92.5 ms


In [23]:
result = mg.select_matcher([dt, rf, svm, lreg, linreg, nb], table=feature_vectors,
                        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
                       target_attr='gold', metric='f1')

time: 181 ms


In [24]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,dt,<magellan.matcher.dtmatcher.DTMatcher object at 0x1161dac88>,5,0.971429,1.000000,0.988764,0.988506,0.967033,0.983146
1,rf,<magellan.matcher.rfmatcher.RFMatcher object at 0x1161dacc0>,5,0.970874,1.000000,0.989247,1.000000,0.977778,0.987580
2,svm,<magellan.matcher.svmmatcher.SVMMatcher object at 0x1161dad68>,5,0.880000,0.925000,0.924731,0.968750,0.971963,0.934089
3,logreg,<magellan.matcher.logregmatcher.LogRegMatcher object at 0x1161dae10>,5,1.000000,1.000000,0.950495,0.976744,0.988764,0.983201
4,linreg,<magellan.matcher.linregmatcher.LinRegMatcher object at 0x1161dae80>,5,0.969072,0.988764,0.989247,0.969697,1.000000,0.983356
5,nb,<magellan.matcher.nbmatcher.NBMatcher object at 0x1161daef0>,5,0.988235,0.945055,1.000000,0.979592,0.981132,0.978803


time: 15 ms
